# Module 5 Assignment

A few things you should keep in mind when working on assignments:

1. Run the first code cell to import modules needed by this assignment before proceeding to problems.
2. Make sure you fill in any place that says `# YOUR CODE HERE`. Do not write your answer anywhere else other than where it says `# YOUR CODE HERE`. Anything you write elsewhere will be removed or overwritten by the autograder.
3. Each problem has an autograder cell below the answer cell. Run the autograder cell to check your answer. If there's anything wrong in your answer, the autograder cell will display error messages.
4. Before you submit your assignment, make sure everything runs as expected. Go to the menubar, select Kernel, and Restart & Run all. If the notebook runs through the last code cell without error message, you've answered all problems correctly.
5. Make sure that you save your work (in the menubar, select File → Save and CheckPoint).

-----

# Run Me First!

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from nose.tools import assert_equal, assert_almost_equal, assert_true, assert_is_instance

# We do this to ignore warnings
import warnings
warnings.filterwarnings("ignore")

-----

# Prepare MPG Data

In this assignment, we will use the mpg dataset to make a regression model. Before we attempt to build a model, we first prepare the data.

Please run the next code cell before proceeding to Problem 1.

-----

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#load MPG dataset
mpg = sns.load_dataset('mpg')
mpg.dropna(inplace=True)
mpg['origin'] = LabelEncoder().fit_transform(mpg.origin)
y = mpg['mpg']
x = mpg[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin']]

# Split data intro training:testing data set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=23)
x_train.sample(2)

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
384,4,91.0,67.0,1965,15.7,82,1
112,4,122.0,85.0,2310,18.5,73,2


---
# Problem 1: Get Feature Ranking by Recursive Feature Extraction

Perform RFE on a Random Forest Regressor and retieve feature rankings.

This problem will use **x** and __y__ created above.

To solve this problem do the following:
- Create a `RandomForestRegressor` estimator. Set `n_estimators` to 100, `random_state` to 23 and accept default values for all other hyperparameters.
- Create a Recursive Feature Estimator `RFE` using the random forest regressor created in step 1 as the `estimator`, set `n_features_to_select ` to 1. Accept default values for other arguments.
- Fit the RFE estimator using **x** and __y__.
- Retrive feature rankings from the `RFE` selector's `ranking_` attribute and assign it to variable **feature_ranking**.

After this problem, there's a new variable **feature_ranking** defined.

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

### BEGIN SOLUTION
rfr = RandomForestRegressor(n_estimators=100, random_state=23)
rfe = RFE(rfr, 1)
rfe.fit(x, y)
feature_ranking = rfe.ranking_
### END SOLUTION

In [4]:
assert_equal(feature_ranking.tolist(), [3, 1, 5, 2, 6, 4, 7])
# Display feature ranking
print('Feature Ranking:')
for var, name in sorted(zip(feature_ranking, x.columns), key=lambda x: x[0]):
    print(f'{name:>12} = {var}')

Feature Ranking:
displacement = 1
      weight = 2
   cylinders = 3
  model_year = 4
  horsepower = 5
acceleration = 6
      origin = 7


---

# Problem 2: Get Accuracy Score of a Random Forest Regressor

This problem will use **x_train, x_test, y_train** and __y_test__ created above.

Your task for this problem is to build and train a `RandomForestRegressor` estimator on mpg data and calcualte the estimator's $R^2$ score.

To solve this problem do the following:
- Create a `RandomForestRegressor` estimator **rfr**. Set `n_estimators` to 100, `random_state` to 23 and accept default values for all other hyperparameters.
- Fit the RandomForestRegressor estimator using x_train and y_train.
- Apply rfr `predict` function on x_test to get predicted mpg, save it as **y_pred**
- Use `r2_score` function with y_test and y_pred to get $R^2$ and assign it to variable **r2**

After this problem, there will be a new variable **r2** defined.

-----

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

### BEGIN SOLUTION
rfr = RandomForestRegressor(n_estimators=100, random_state=23)
rfr.fit(x_train, y_train)
y_pred = rfr.predict(x_test)
r2 = r2_score(y_test, y_pred)
### END SOLUTION

In [6]:
assert_almost_equal(r2, 0.8441667341695959, msg="R2 score is not correct")
print(f'R2 score: {r2:4.2f}')

R2 score: 0.84


---

# Problem 3: Get the Cross Validation Scores

Get the cross validation scores for a random forest regressor.

This problem will use **x** and __y__ created above.

To solve this problem do the following:
- Create a `RandomForestRegressor` estimator. Set `n_estimators` to 100, `random_state` to 23 and accept default values for all other hyperparameters.
- Create `KFold` iterator. Set `n_splits` to 5 and `random_state` to 23.
- Calculate cross validation scores using `cross_val_score` function with the random forest regressor, x, y and the `KFold` iterator. Assign scores to variable **cv_scores**.

After this problem, there's a new variable **cv_scores** defined.

-----

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

### BEGIN SOLUTION
rfr = RandomForestRegressor(n_estimators=100, random_state=23)
skf = KFold(n_splits=5, random_state=2)
cv_scores = cross_val_score(rfr, x, y, cv=skf)
### END SOLUTION

In [8]:
assert_almost_equal(cv_scores[0], 0.88705262, msg='Cross validation scores are not correct')
assert_almost_equal(cv_scores[2], 0.85913104, msg='Cross validation scores are not correct')
print(f"Average Cross Validation Score: {np.mean(cv_scores):4.2f}")

Average Cross Validation Score: 0.75


-----

# Prepare Breast Cancer Data

For next 2 problems we will use the breast cancer dataset. Before we attempt to build models, we first prepare the data.

Please run the next two code cells before proceeding to Problem 4.

-----

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Load breast cancer dataset
df = pd.read_csv('data/breast-cancer-wisconsin.csv')
#reduce data size
df = df.sample(200, random_state=23)
label = df['class']
data = df[['clump thickness', 'uniformity cell size', 'uniformity cell shape', 'marginal adhesion', 'epithelial cell size', 'bare nuclei', 'bland chromatin', 'normal nucleoli', 'mitoses']]

---

# Problem 4: Find Best Estimator with Grid Search

Conducts grid search cross validation on the random forest classifier and get the best estimator.

This problem will use **data** and __label__ created above.

To solve this problem do the following:
- Create a `RandomForestClassifier` estimator. Set `random_state` to 23.
- Create a `StratifiedKFold` iterator with `n_splits` equals 5 and `random_state` eaquals 23.
- Create variable **estimators** with value `[20, 40, 60, 80, 100]`
- Create parameter dictionary **params** and set key to `'n_estimators'` and value to the list `estimators` created in previous step.
- Create `GridSearchCV` object:
 - set `estimator` to the random forest classifier estimator
 - set `param_grid` to the parameter dictionary
 - set `cv` to the stratified k-fold iterator
- Fit the `GridSearchCV` object created in previous step using **data** and __label__.
- Retrieve the best esitimator from the `GridSearchCV` object's `best_estimator_` attribute and assign it to variable **best_estimator_gs**.
- Retrieve the best cross validation score from the `GridSearchCV` object's `best_score_` attribute and assign it to variable **best_score_gs**

After this problem, there will be two new variables **best_estimator_gs**, __best_score_gs__ defined.

-----

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

### BEGIN SOLUTION
rfc = RandomForestClassifier(random_state=23)
n_estimators = [20, 40, 60, 80, 100]
skf = StratifiedKFold(n_splits=5, random_state=23)

# Create a dictionary of hyperparameters and values
params = {'n_estimators':n_estimators}

# Create grid search cross validator
gse = GridSearchCV(estimator=rfc, param_grid=params, cv=skf)

# Fit estimator
gse.fit(data, label)

best_estimator_gs = gse.best_estimator_
best_score_gs = gse.best_score_
### END SOLUTION

In [11]:
assert_equal(best_estimator_gs.get_params()["n_estimators"], 40, msg="Best estimator is not correct.")
assert_almost_equal(best_score_gs, 0.975, places=3, msg="Best score is not correct.")

print(f'Best n_estimators={best_estimator_gs.get_params()["n_estimators"]}')
print(f'Best CV Score = {best_score_gs:5.3f}')

Best n_estimators=40
Best CV Score = 0.975


---

# Problem 5: Find Best Estimator with Random Grid Search

Conducts random grid search cross validation on the random forest classifier and get the best estimator.

This problem will use **data** and __label__ created above.

To solve this problem do the following:
- Create a `RandomForestClassifier` estimator. Set `random_state` to 23.
- Create a `StratifiedKFold` iterator with `n_splits` equals 5 and `random_state` eaquals 23.
- Create variable **estimators** with value `range(20,100)`
- Create variable __weights__ with value `[None, 'balanced']`
- Create parameter dictionary **params** with two keys, `'n_estimators'` and `'class_weight'`, and two list values `estimators` and `weights` created in previous step.
- Create `RandomizedSearchCV` object:
 - set `estimator` to the random forest classifier estimator
 - set `param_distributions` to the parameter dictionary
 - set `cv` to the stratified k-fold iterator
 - set `n_iter` to 5
 - set `random_state` to 23
- Fit the `RandomizedSearchCV` object created in previous step using **data** and __label__.
- Retrieve the best esitimator from the `RandomizedSearchCV` object's `best_estimator_` attribute and assign it to variable **best_estimator_rgs**.
- Retrieve the best cross validation score from the `RandomizedSearchCV` object's `best_score_` attribute and assign it to variable **best_score_rgs**

After this problem, there will be two new variable **best_estimator_rgs** and __best_score_rgs__ defined.

**Note**: If you get warning messages after running your code, just ignore them.

-----

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

### BEGIN SOLUTION
rfc = RandomForestClassifier(random_state=23)
skf = StratifiedKFold(n_splits=5, random_state=23)

estimators = range(20, 100)
weights = [None, 'balanced']
# Create a dictionary of hyperparameters and values
params = {'n_estimators':estimators, 'class_weight':weights}

# Create grid search cross validator
rgse = RandomizedSearchCV(estimator=rfc, param_distributions=params, n_iter=5, cv=skf, random_state=23)

# Fit estimator
rgse.fit(data, label)

best_estimator_rgs = rgse.best_estimator_
best_score_rgs = rgse.best_score_
### END SOLUTION

/Users/lindenlu/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [13]:
assert_equal(best_estimator_rgs.get_params()["n_estimators"], 49, msg="Best n_estimator is not correct.")
assert_equal(best_estimator_rgs.get_params()["class_weight"], 'balanced', msg="Best class_weight is not correct.")
print(f'Best n_estimators={best_estimator_rgs.get_params()["n_estimators"]}')
print(f'Best class_weight={best_estimator_rgs.get_params()["class_weight"]}')
print(f'Best CV Score = {best_score_rgs:5.3f}')

Best n_estimators=49
Best class_weight=balanced
Best CV Score = 0.975
